In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Loading Data

In [11]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [12]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked


# Feature Engineering

In [13]:
# def diabetes_indicator(avg_glucose_level):
#     if avg_glucose_level <= 99:
#         return "normal"
#     elif avg_glucose_level >= 100 and avg_glucose_level <= 125:
#         return "prediabetic"
#     elif avg_glucose_level <= 200:
#         return "type1"
#     else:
#         return "type2"

# df.avg_glucose_level.map(diabetes_indicator).value_counts()

In [14]:
# df["diabetes"] = df.avg_glucose_level.map(diabetes_indicator)
# df.head(3)

# One hot encoding

In [15]:
df = pd.get_dummies(df)
df.head(3)

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,28.0,0,0,79.53,31.1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,1,0
1,33.0,0,0,78.44,23.9,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,42.0,0,0,103.00,40.3,1,0,0,0,1,...,0,1,0,0,1,0,1,0,0,0


# Preprocessing v2

In [22]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [23]:
inputs = layers.Input(shape=(21,))
x = layers.Dense(512, activation="relu")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(448, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

x = layers.Dense(32, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.7)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

keras_model = keras.Model(inputs=inputs, outputs=outputs)

optim = keras.optimizers.Adam(learning_rate= 0.0035)

keras_model.compile(optimizer=optim,
                   loss=keras.losses.binary_crossentropy,
                   metrics=[keras.metrics.AUC()])

In [24]:
history = keras_model.fit(X, y, validation_split=0.2, epochs=48)

Epoch 1/48
383/383 [==============================] - 6s 10ms/step - loss: 0.3187 - auc_2: 0.6412 - val_loss: 0.1232 - val_auc_2: 0.8858
Epoch 2/48
383/383 [==============================] - 3s 9ms/step - loss: 0.1540 - auc_2: 0.8008 - val_loss: 0.1173 - val_auc_2: 0.8847
Epoch 3/48
383/383 [==============================] - 4s 9ms/step - loss: 0.1502 - auc_2: 0.8194 - val_loss: 0.1136 - val_auc_2: 0.8932
Epoch 4/48
383/383 [==============================] - 4s 10ms/step - loss: 0.1446 - auc_2: 0.8324 - val_loss: 0.1143 - val_auc_2: 0.8885
Epoch 5/48
383/383 [==============================] - 4s 10ms/step - loss: 0.1422 - auc_2: 0.8490 - val_loss: 0.1191 - val_auc_2: 0.8852
Epoch 6/48
383/383 [==============================] - 3s 9ms/step - loss: 0.1412 - auc_2: 0.8471 - val_loss: 0.1181 - val_auc_2: 0.8883
Epoch 7/48
383/383 [==============================] - 3s 9ms/step - loss: 0.1430 - auc_2: 0.8377 - val_loss: 0.1154 - val_auc_2: 0.8913
Epoch 8/48
383/383 [=========================

In [28]:
y_pred = keras_model.predict(test_new)[:, 0]
y_pred

array([0.02983281, 0.1416511 , 0.00030428, ..., 0.00027686, 0.00157326,
       0.0001955 ], dtype=float32)

In [29]:
# since we dropped id column from our test dataframe earlier but we need it for submission file so we're reloading it
test_idx = pd.read_csv(BASE_DIR / "test.csv").id
test_idx

0        15304
1        15305
2        15306
3        15307
4        15308
         ...  
10199    25503
10200    25504
10201    25505
10202    25506
10203    25507
Name: id, Length: 10204, dtype: int64

In [30]:
submission = pd.DataFrame(data={"id": test_idx, "stroke": y_pred})
submission.head()

,id,stroke
0,15304,0.029833
1,15305,0.141651
2,15306,0.000304
3,15307,0.059572
4,15308,0.006907


In [31]:
submission.to_csv("submission.csv", index=False)